# LSTM Preprocessing

### Obsah<a class='anchor' id='top'></a>
* [Globálne parametre](#1)
* [Načítanie dát](#2)
* [Rozdelenie dát na testovaciu a trénovaciu sadu](#3)
* [Vytvorenie sekvencií](#4)

In [1]:
import json
import warnings

import joblib
import numpy as np
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

### Globálne parametre<a class='anchor' id='1'></a>

In [2]:
# Globálne konfiguračné premenné

# Cesty k súborom    
X_PATH = './preprocessed_data/X_scaled.npy'
Y_PATH = './preprocessed_data/y_labels.npy'
SCALER_PATH = './scaler/scaler.joblib'
FEATURES_PATH = './json/selected_features.json'  
UC_MAP_PATH = './json/uc_map.json'

# Názov stĺpca s cieľovou premennou
LABEL_COLUMN = 'current_uc'       

# Parametre pre rozdelenie dát
TEST_SIZE = 0.2                   
SEED = 42                                       

# Parametre pre LSTM model
SEQUENCE_LENGTH = 60                                

<div class="alert alert-warning">
<b>Súbory z predošlých krokov:</b> Je potrebné zabezpečiť, že jupyter notebooky z predošlých krokov sú spustené a súbory sú uložené v správnych priečinkoch. V prípade, že sa niektorý z predchádzajúcich krokov nezobrazil, je potrebné ho spustiť pred týmto krokom.
</div>

### Načítanie predspracovaných dát <a class='anchor' id='2'></a>

In [3]:
with open(FEATURES_PATH, 'r') as f:
    selected_features = json.load(f)['features']

In [4]:
X_scaled = np.load(X_PATH)
y = np.load(Y_PATH)
scaler = joblib.load(SCALER_PATH)

with open(UC_MAP_PATH, "r") as f:
    uc_map = json.load(f)

### Rozdelenie dát na trénovaciu a testovaciu množinu <a class='anchor' id='3'></a>

In [5]:
def create_sequences(X, y, seq_len):

    """Create sequences of data with given length for LSTM input."""

    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

#### Vytvorenie sekvencií <a class='anchor' id='4'></a>

In [6]:
# Create sliding window sequences
X_seq, y_seq = create_sequences(X_scaled, y, SEQUENCE_LENGTH)

# Then split
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=TEST_SIZE, random_state=SEED, stratify=y_seq
)

In [7]:
print(f"✅ X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"✅ X_test: {X_test.shape}, y_test: {y_test.shape}")

✅ X_train: (34984, 60, 8), y_train: (34984,)
✅ X_test: (8747, 60, 8), y_test: (8747,)


In [8]:
# Save the X_train and y_train, X_test and y_test arrays
np.save('preprocessed_data/X_train.npy', X_train)
np.save('preprocessed_data/y_train.npy', y_train)
np.save('preprocessed_data/X_test.npy', X_test)
np.save('preprocessed_data/y_test.npy', y_test)

<div class="alert alert-success">
<b>Preprocessing dát:</b> Dáta sú predspracované a rozdelené na trénovaciu a testovaciu množinu. Sekvencie sú vytvorené z trénovacej množiny a testovacia množina je pripravená na hodnotenie modelu.
</div>